# app

In [1]:
%cd /workspace/twint/app
!pip install -e .

/workspace/twint/app
Obtaining file:///workspace/twint/app
  Running setup.py develop for app
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


# scraper

In [ ]:
%cd /workspace/twint/app
!python -m app.main hydra.verbose=true

/workspace/twint/app
[2020-06-01 16:29:04,674][HYDRA] Installed Hydra Plugins
[2020-06-01 16:29:04,674][HYDRA] ***********************
[2020-06-01 16:29:04,674][HYDRA] 	SearchPathPlugin:
[2020-06-01 16:29:04,674][HYDRA] 	-----------------
[2020-06-01 16:29:04,676][HYDRA] 	Sweeper:
[2020-06-01 16:29:04,676][HYDRA] 	--------
[2020-06-01 16:29:04,678][HYDRA] 		BasicSweeper
[2020-06-01 16:29:04,678][HYDRA] 	Launcher:
[2020-06-01 16:29:04,678][HYDRA] 	---------
[2020-06-01 16:29:04,680][HYDRA] 		BasicLauncher
[2020-06-01 16:29:04,680][HYDRA] 
[2020-06-01 16:29:04,680][HYDRA] Hydra config search path
[2020-06-01 16:29:04,680][HYDRA] ************************
[2020-06-01 16:29:04,681][HYDRA] | Provider | Search path               |
[2020-06-01 16:29:04,681][HYDRA] ----------------------------------------
[2020-06-01 16:29:04,681][HYDRA] | hydra  | pkg://hydra.conf          |
[2020-06-01 16:29:04,681][HYDRA] | main   | /workspace/twint/app/app  |
[2020-06-01 16:29:04,681][HYDRA] 
[2020-06-01 16

[2020-06-01 16:29:36,569][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484658 HTTP/1.1" 200 2844
[2020-06-01 16:29:36,576][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484663 HTTP/1.1" 200 2529
[2020-06-01 16:29:36,582][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484662 HTTP/1.1" 200 3235
[2020-06-01 16:29:36,590][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484661 HTTP/1.1" 200 4872
[2020-06-01 16:29:36,597][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484660 HTTP/1.1" 200 2735
[2020-06-01 16:29:36,614][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484630 HTTP/1.1" 200 3013
[2020-06-0

[2020-06-01 16:30:07,559][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484462 HTTP/1.1" 200 2567
[2020-06-01 16:30:07,571][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484499 HTTP/1.1" 200 2748
[2020-06-01 16:30:07,582][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484473 HTTP/1.1" 200 3055
[2020-06-01 16:30:07,594][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484497 HTTP/1.1" 200 2897
[2020-06-01 16:30:07,604][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484496 HTTP/1.1" 200 2003
[2020-06-01 16:30:07,612][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484482 HTTP/1.1" 200 2851
[2020-06-0

[2020-06-01 16:31:09,005][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484186 HTTP/1.1" 200 1723
[2020-06-01 16:31:09,012][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484116 HTTP/1.1" 200 2915
[2020-06-01 16:31:09,020][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484185 HTTP/1.1" 200 1725
[2020-06-01 16:31:09,026][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4483832 HTTP/1.1" 200 2103
[2020-06-01 16:31:09,032][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484111 HTTP/1.1" 200 2654
[2020-06-01 16:31:09,043][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4483079 HTTP/1.1" 200 3998
[2020-06-0

[2020-06-01 16:31:39,910][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484028 HTTP/1.1" 200 2956
[2020-06-01 16:31:39,916][app.scrapers.cnyes][INFO] - scraped: https://news.cnyes.com/api/v3/news/category/headline?limit=30&startAt=1588608000&endAt=1591113599&page=6
[2020-06-01 16:32:11,050][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484031 HTTP/1.1" 200 2713
[2020-06-01 16:32:11,061][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4484029 HTTP/1.1" 200 2662
[2020-06-01 16:32:11,067][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4483956 HTTP/1.1" 200 2702
[2020-06-01 16:32:11,074][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4483829 HTTP/1.1" 200 2330
[2020-0

...

[2020-06-01 17:07:20,762][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472497 HTTP/1.1" 200 4168
[2020-06-01 17:07:20,779][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472717 HTTP/1.1" 200 3134
[2020-06-01 17:07:20,804][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472666 HTTP/1.1" 200 3340
[2020-06-01 17:07:20,812][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472719 HTTP/1.1" 200 2478
[2020-06-01 17:07:20,824][app.scrapers.cnyes][INFO] - scraped: https://news.cnyes.com/api/v3/news/category/headline?limit=30&startAt=1588608000&endAt=1591113599&page=74
[2020-06-01 17:07:51,497][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472716 HTTP/1.1" 200 2198
[2020-

[2020-06-01 17:08:23,025][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472478 HTTP/1.1" 200 3145
[2020-06-01 17:08:23,031][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472233 HTTP/1.1" 200 2786
[2020-06-01 17:08:23,038][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471940 HTTP/1.1" 200 2510
[2020-06-01 17:08:23,048][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472213 HTTP/1.1" 200 2652
[2020-06-01 17:08:23,054][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472369 HTTP/1.1" 200 2905
[2020-06-01 17:08:23,060][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4472364 HTTP/1.1" 200 2091
[2020-06-0

[2020-06-01 17:09:24,744][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471876 HTTP/1.1" 200 2636
[2020-06-01 17:09:24,760][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471875 HTTP/1.1" 200 2692
[2020-06-01 17:09:24,768][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471869 HTTP/1.1" 200 2054
[2020-06-01 17:09:24,774][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471717 HTTP/1.1" 200 3167
[2020-06-01 17:09:24,779][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471820 HTTP/1.1" 200 2420
[2020-06-01 17:09:24,786][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471836 HTTP/1.1" 200 2608
[2020-06-0

[2020-06-01 17:09:57,267][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471688 HTTP/1.1" 200 2490
[2020-06-01 17:09:57,284][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471350 HTTP/1.1" 200 2488
[2020-06-01 17:09:57,320][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471687 HTTP/1.1" 200 2704
[2020-06-01 17:09:57,336][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471691 HTTP/1.1" 200 3063
[2020-06-01 17:09:57,348][app.scrapers.cnyes][INFO] - scraped: https://news.cnyes.com/api/v3/news/category/headline?limit=30&startAt=1588608000&endAt=1591113599&page=79
[2020-06-01 17:10:28,024][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471690 HTTP/1.1" 200 2494
[2020-

[2020-06-01 17:11:00,149][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471339 HTTP/1.1" 200 1988
[2020-06-01 17:11:00,185][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471289 HTTP/1.1" 200 2134
[2020-06-01 17:11:00,353][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471354 HTTP/1.1" 200 2727
[2020-06-01 17:11:00,379][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471258 HTTP/1.1" 200 2603
[2020-06-01 17:11:00,405][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471154 HTTP/1.1" 200 2616
[2020-06-01 17:11:00,412][urllib3.connectionpool][DEBUG] - http://es:9200 "GET /news_page/_doc/https%3A%2F%2Fnews.cnyes.com%2Fnews%2Fid%2F4471264 HTTP/1.1" 200 2739
[2020-06-0

rss sources:
https://tw.stock.yahoo.com/rss_index.html

# Twint

In [1]:
# %cd /workspace/twint
# !pip install e .
# !twint -u CNBC
# !pip install -U fake-useragent

Requirement already up-to-date: fake-useragent in /usr/local/lib/python3.7/site-packages (0.1.11)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
import nest_asyncio
nest_asyncio.apply()
import twint

c = twint.Config()
c.Username = "CNBC"
c.Elasticsearch = "http://es:9200"
c.Until='2015-01-01 00:00:00'

# c.Search = "fruit"
twint.run.Search(c)

[+] Indexing to Elasticsearch @ http://es:9200
..................................................................................................

KeyboardInterrupt: 

# elasticsearch

dump: https://github.com/taskrabbit/elasticsearch-dump  

```
elasticdump \
  --input=http://es:9200/twinttweets \
  --output=./twinttweets_mapping_20200503.json \
  --type=mapping
elasticdump \
  --input=http://es:9200/twinttweets \
  --output=./twinttweets_index_20200503.json \
  --type=data

elasticdump \
  --input=http://es:9200/twinttweets \
  --output=$ \
  | gzip > ./twinttweets_index_20200504.json.gz
```